# SWMAL Exercise

## Advanced CNN using Roboflow

In this exercise we explore object detection machine learning algorithms, using Roboflow and YOLOv8 API. YOLOv8 is an api developed by Ultralytics, which is designed for CNN models. It is a successor of YOLOv3 and YOLOv5, and has proven itself to be a powerful framework for tasks such as object detection and classification. Because of this we will use it for the most glorious purpose imaginable: detecting cats! And if you hate cats, we can use it to crop them out of the pictures...

## Creating a dataset to work with

In order to detect cats properly, one must first create a dataset of images which the model can work with. We will start with cloning an appropriate project from https://universe.roboflow.com/ . For this project we chose a project which already has 50 images to work with, and luckily for us they are annotated. Cloning the images is quite straightforward - you select the project to clone from, choose the images to clone and your Roboflow workspace/project where the images will be placed.

![cloning in roboflow](cnn2_Imgs/roboflow_cloning.png)





In this case, the images which we have cloned are already annotated: there are bounding boxes around the cats. This is helpful, because it will spare us the time to go through each of these images individually and annotate them ourselves. However, to illustrate how such process is done we decided to upload a few images of our own, and annotate them.

![roboflow annotations](cnn2_imgs/roboflow_annotation.png)




We then generate a new version of the dataset. This step performs the Train/Test split and some preprocessing on the images, notably by downsizing them since ML models tend to work better on smaller pixel resolution images.

![roboflow version generation](cnn2_imgs/roboflow_version.png)

This also creates a code snippet that downloads the generated dataset.


## Yolo CLI and SDK intro

To make sure that everything is setup correctly we will go through the tutorial on https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/ . This also introduces us briefly to YOLO SDK in python.

In [10]:
# Checks that we have access to the GPU
! nvidia-smi

from IPython import display
display.clear_output()

# Check that yolo is installed properly by running this CLI command. If no exception is thrown, we are golden.
! yolo mode="track"
# YOLO installs some missing dependencies on its own when running via CLI! Neat!






WARNING ⚠️ 'model' is missing. Using default 'model=yolov8n.pt'.
WARNING ⚠️ 'source' is missing. Using default 'source=/home/mszymanek/anaconda3/envs/homl3/lib/python3.10/site-packages/ultralytics/assets'.
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Core(TM) i7-7600U 2.80GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/2 /home/mszymanek/anaconda3/envs/homl3/lib/python3.10/site-packages/ultralytics/assets/bus.jpg: 640x480 3 persons, 1 bus, 149.9ms
image 2/2 /home/mszymanek/anaconda3/envs/homl3/lib/python3.10/site-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 114.0ms
Speed: 3.2ms preprocess, 131.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5
💡 Learn more at https://docs.ultralytics.com/modes/track


In [11]:
# test SDK
from ultralytics import YOLO
from PIL import Image

# Download and show the dog picture
! curl https://media.roboflow.com/notebooks/examples/dog.jpeg -o ./dog.jpeg
img = Image.open("./dog.jpeg")
img.show()

# Test on the tutorial example with a dog. pt = model is pretrained
model = YOLO('yolov8n.pt')
results = model.predict(source="https://media.roboflow.com/notebooks/examples/dog.jpeg", conf=0.25)




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  103k  100  103k    0     0   316k      0 --:--:-- --:--:-- --:--:--  315k


Aborted

Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 /home/mszymanek/UNI_autumn_2023/machineLearning/swmal_grp10/O3/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 112.9ms
Speed: 2.4ms preprocess, 112.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)


The algorithm found even the blurred car in the picture, which I myself have missed. Quite impressive.

It's time to train and test a YOLO model on our cat dataset. The first step is to load the data from Roboflow.

## Loading and training on data from Roboflow

We will use the code generated earlier by Roboflow to download the data from it. It uses roboflows own package and api to download data securely.

In [12]:
# If you haven't installed roboflow package:
# ! pip install roboflow
 
from roboflow import Roboflow
rf = Roboflow(api_key="RcblOwaZETwq19a8fyov")
project = rf.workspace("swmal10").project("catfinder-hh2e7")
dataset = project.version(1).download("yolov8")

# You must change the paths in project/data.yaml : all paths are relative to data.yaml directory, but roboflow mistakingly generates paths with project dir... and test path goes back for no reason


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.208, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to CatFinder-1 in yolov8:: 100%|██████████| 122/122 [00:00<00:00, 3332.13it/s]


In [13]:
from ultralytics import YOLO
import os
import platform
newmodel = YOLO("yolov8n.pt")

# Find the directory to the data.yaml file. We need to change its contents because Roboflow
# puts wrong paths in data.yaml. This might be due to incorrect versions? Roboflow
# complains about yolo lib being too new...



def fix_my_yaml(path):
    print(path)
    # Replace the incorrect paths in .yaml
    lines = []
    with open (path) as f:
        for line in f:
            lines.append(line)
            # print(line)

    with open (datapath, "w") as f:
        for s in lines:
            if "../test" in s:
                s = s.replace("../", "")
                f.write(s)
            elif "train:" in s:
                s = s.replace("CatFinder-1/", "")
                f.write(s)
            elif "val:" in s:
                s = s.replace("CatFinder-1/", "")
                f.write(s)
            else:
                f.write(s)
    print("Fixed data.yaml in ", datapath)

# Don't run on Windows
if platform.system() == "Linux":
    ! find / -iname "CatFinder-1"  2>&1 | grep -v "Permission denied" | grep -v "Invalid argument" > path.txt
    with open("path.txt") as f: s = f.readline()[:-1] # -1 because find returns \n at the end
    os.remove("path.txt")
    datapath = s + "/data.yaml"
    fix_my_yaml(datapath)
    res = newmodel.train(data=datapath)

elif platform.system() == "Windows":
    # Please insert your own path and make sure the .yaml file is in order
    res = newmodel.train(data="C:\\UNI_2023\\ml\\swmal_grp10\\O3\\CatFinder-1\\data.yaml")

# The more epochs, the better: I run 100 on my desktop but 10 on my laptop
# If on gpu cluster/some other server you can increase epochs


# This crashed on my computer every time for some reason...
# The reason was OBVIOUSLY one of the dependencies. Probably matplotlib or scipy (called by seaborn, but seaborn was not the problem)
# Solved by upgrading via python -m pip install <package> --upgrade
print(res)


/home/mszymanek/UNI_autumn_2023/machineLearning/swmal_grp10/O3/CatFinder-1/data.yaml
Fixed data.yaml in  /home/mszymanek/UNI_autumn_2023/machineLearning/swmal_grp10/O3/CatFinder-1/data.yaml


Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Core(TM) i7-7600U 2.80GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mszymanek/UNI_autumn_2023/machineLearning/swmal_grp10/O3/CatFinder-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None,

KeyboardInterrupt: 

All the results are stored in runs/train/ directory and include labeled images as well as a lot of statistical data about the models performance. Among those there is the precision-recall curve, which rightly trends to zero over time. 

![PR-curve](cnn2_imgs/PRcurve.png)